In [99]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
import seaborn as sns
import cv2

In [100]:
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image
from PIL import Image

In [101]:
# sample_file=r'/content/drive/My Drive/WORK/merge_NR5G_DL.csv'
# sample_file=r'/content/drive/My Drive/WORK/1910_2003_5G_NQI_preprocessed.csv'
sample_file=r'/content/drive/My Drive/WORK/New_road_train_Month6_NR5G_DL.csv'
dataCombine = pd.read_csv(sample_file, encoding='ISO-8859-1')

In [102]:
# ctq_anaysis_merge_list= dataCombine[['call_id','serving_network','call_type','total_call_state','nr5g_Rsrp','nr5g_Rsrq','nr5g_Sinr','nr5g_TxPwrTotal_Actual','nr5g_Pdsch_Bler','nr5g_pc_pusch_bler','nr5g_DlMcs_Ant0_Avg','data_ex_rx_thravg','data_ftp_rx_tp']]
ctq_anaysis_merge_list= dataCombine[['call_id','serving_network','sampled_time' , 'call_type','total_call_state','nr5g_Rsrp','nr5g_Sinr','nr5g_TxPwrTotal_Actual','nr5g_Pdsch_Bler','nr5g_pc_pusch_bler','nr5g_DlMcs_Ant0_Avg','nr5g_Rx_NumOfRb_Avg','nr5g_UlMcs_Avg','data_ex_rx_thravg','data_ftp_rx_tp']]

In [103]:
ctq_anaysis_merge_list = ctq_anaysis_merge_list[ctq_anaysis_merge_list['total_call_state'] != 'Idle']
ctq_anaysis_merge_list = ctq_anaysis_merge_list[ctq_anaysis_merge_list['total_call_state'] != 'Setup']
ctq_anaysis_merge_list = ctq_anaysis_merge_list[ctq_anaysis_merge_list['total_call_state'] != 'In-Traffic']
ctq_anaysis_merge_list = ctq_anaysis_merge_list[ctq_anaysis_merge_list['call_type'] != 'U']
ctq_anaysis_merge_list.reset_index(drop=True, inplace=True)
ctq_anaysis_merge_list['result'] = 0
# ctq_anaysis_merge_list.loc[ctq_anaysis_merge_list['data_ex_rx_thravg'] < 12, 'result'] = 0
ctq_anaysis_merge_list.loc[ctq_anaysis_merge_list['data_ftp_rx_tp'] < 200, 'result'] = 1

In [104]:
ctq_anaysis_merge_list.dropna(subset = ctq_anaysis_merge_list.columns, axis = 0, inplace = True)
ok_frame = ctq_anaysis_merge_list.loc[ctq_anaysis_merge_list['result'] == 0]
fail_frame = ctq_anaysis_merge_list.loc[ctq_anaysis_merge_list['result'] == 1]
ok_frame = ok_frame[ok_frame['data_ftp_rx_tp'] > 600]

In [105]:
cut_list = ok_frame.append(fail_frame)
cut_list.reset_index(drop=True, inplace=True)

In [13]:
cut_list['result'].value_counts()

1    69902
0    66888
Name: result, dtype: int64

In [113]:
cut_list.loc[cut_list['result'] == 1, : ].describe()

,nr5g_Rsrp,nr5g_Sinr,nr5g_TxPwrTotal_Actual,nr5g_Pdsch_Bler,nr5g_pc_pusch_bler,nr5g_DlMcs_Ant0_Avg,nr5g_Rx_NumOfRb_Avg,nr5g_UlMcs_Avg,data_ex_rx_thravg,data_ftp_rx_tp,result
count,69902.000000,69902.000000,69902.000000,69902.000000,69902.000000,69902.000000,69902.000000,69902.000000,69902.000000,69902.000000,69902.0
mean,-84.159991,5.724289,16.134029,10.395062,13.934417,12.434063,208.580227,17.954306,158.753382,93.761211,1.0
std,9.161818,6.837977,9.042858,7.136573,11.920426,5.343424,60.739500,5.923266,113.050697,62.060184,0.0
min,-130.650000,-21.860000,-42.500000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,1.0
25%,-90.030000,0.580000,10.300000,7.072000,9.555000,8.806000,185.000000,14.699000,74.085750,39.536000,1.0
50%,-83.620000,5.160000,22.000000,9.677000,12.286000,12.629000,225.000000,18.735500,137.933000,91.336000,1.0
75%,-77.910000,10.450000,23.000000,13.067000,14.711000,16.057750,255.000000,22.094000,223.247000,148.700000,1.0
max,-47.100000,24.990000,23.000000,100.000000,100.000000,26.842000,273.000000,29.684000,732.695000,199.998000,1.0


In [14]:
train_x = cut_list.drop(['call_id','sampled_time' ,'serving_network','call_type','total_call_state','result','data_ftp_rx_tp','data_ex_rx_thravg'],axis=1)
train_y = cut_list['result']

In [15]:
TIME_STEPS = 1
INPUT_DIM  = train_x.shape[1]
x_scale = minmax_scale(train_x, axis=0, copy=True)
y_scale = minmax_scale(train_y, axis=0, copy=True)
x_scale=x_scale.reshape(-1,TIME_STEPS,INPUT_DIM,1)

In [16]:
K.clear_session()
model1 = Sequential()
model1.add(Conv2D(128, 1, input_shape=(TIME_STEPS, INPUT_DIM, 1),activation='relu'))
model1.add(Conv2D(128, 1,activation='relu'))
model1.add(BatchNormalization())
model1.add(Conv2D(256, 1,activation='relu'))
model1.add(Conv2D(256, 1,activation='relu'))
model1.add(BatchNormalization())
model1.add(Conv2D(512, 1,activation='relu'))
model1.add(Conv2D(512, 1,activation='relu'))
model1.add(Flatten())
model1.add(Dense(4096, activation='relu'))
model1.add(Dense(1024, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(2, activation='softmax'))
model1.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
               metrics=['accuracy'], optimizer=Adam())

In [88]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model1.fit(x=x_scale, y=train_y, validation_split=0.2, epochs=10000, callbacks=[early_stop])

Epoch 1/10000
3420/3420 [==============================] - 59s 17ms/step - loss: 0.0954 - accuracy: 0.9688 - val_loss: 0.0400 - val_accuracy: 0.9838
Epoch 2/10000
3420/3420 [==============================] - 58s 17ms/step - loss: 0.0921 - accuracy: 0.9696 - val_loss: 0.0754 - val_accuracy: 0.9700
Epoch 3/10000
3420/3420 [==============================] - 58s 17ms/step - loss: 0.0917 - accuracy: 0.9693 - val_loss: 0.0909 - val_accuracy: 0.9656
Epoch 4/10000
3420/3420 [==============================] - 59s 17ms/step - loss: 0.0910 - accuracy: 0.9695 - val_loss: 0.0377 - val_accuracy: 0.9846
Epoch 5/10000
3420/3420 [==============================] - 58s 17ms/step - loss: 0.0889 - accuracy: 0.9706 - val_loss: 0.2153 - val_accuracy: 0.9296
Epoch 6/10000
3420/3420 [==============================] - 59s 17ms/step - loss: 0.0880 - accuracy: 0.9710 - val_loss: 0.1249 - val_accuracy: 0.9531
Epoch 7/10000
3420/3420 [==============================] - 59s 17ms/step - loss: 0.0868 - accuracy: 0.9711

In [18]:
model1.save(r'/content/drive/My Drive/WORK/New_featureanal.h5')

In [19]:
def load_image(path, target_size=(224, 224)):
    x = image.load_img(path, target_size=target_size)
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def generate_gradcam(img_tensor, model, class_index, activation_layer):
    model_input = model1.input

    # y_c : class_index에 해당하는 CNN 마지막 layer op(softmax, linear, ...)의 입력
    y_c = model.outputs[0].op.inputs[0][0, class_index]

    # A_k: activation conv layer의 출력 feature map
    A_k = model.get_layer(activation_layer).output

    # model의 입력에 대해서,
    # activation conv layer의 출력(A_k)과
    # 최종 layer activation 입력(y_c)의 A_k에 대한 gradient,
    # 모델의 최종 출력(prediction) 계산
    grad_model = tf.keras.models.Model([model1.inputs], [model1.get_layer(conv_name).output, model1.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model([img_tensor])
        loss = predictions[:, 1]
    conv_output = conv_outputs[0]
    grad_val = tape.gradient(loss, conv_outputs)[0]
    
    # get_output = K.function([model_input], [A_k, K.gradients(y_c, A_k)[0]])
    # [conv_output, grad_val] = get_output([img_tensor])

    # batch size가 포함되어 shape가 (1, width, height, k)이므로
    # (width, height, k)로 shape 변경
    # 여기서 width, height는 activation conv layer인 A_k feature map의 width와 height를 의미함
    # conv_output = conv_output[0]
    # grad_val = grad_val[0]

    # global average pooling 연산
    # gradient의 width, height에 대해 평균을 구해서(1/Z) weights(a^c_k) 계산
    weights = np.mean(grad_val, axis=(0, 1))

    # activation conv layer의 출력 feature map(conv_output)과
    # class_index에 해당하는 weights(a^c_k)를 k에 대응해서 weighted combination 계산

    # feature map(conv_output)의 (width, height)로 초기화
    grad_cam = np.zeros(dtype=np.float32, shape=conv_output.shape[0:2])
    # print(weights)
    for k, w in enumerate(weights):
        grad_cam += w * conv_output[:, :, k]

    # 계산된 weighted combination 에 ReLU 적용
    grad_cam = np.maximum(grad_cam, 0)
    # print(grad_cam)
    return grad_cam, weights

In [20]:
test_file = r'/content/drive/My Drive/WORK/202007_NQI_도로_수도권_KT_NR5G_FTP_DL.csv'
testCombine = pd.read_csv(sample_file, encoding='ISO-8859-1')

In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_x)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [22]:
test_list = testCombine[['call_id','serving_network','sampled_time' , 'call_type','total_call_state','nr5g_Rsrp','nr5g_Sinr','nr5g_TxPwrTotal_Actual','nr5g_Pdsch_Bler','nr5g_pc_pusch_bler','nr5g_DlMcs_Ant0_Avg','nr5g_Rx_NumOfRb_Avg','nr5g_UlMcs_Avg','data_ex_rx_thravg','data_ftp_rx_tp']]
test_list = test_list[test_list['total_call_state'] != 'Idle']
test_list = test_list[test_list['total_call_state'] != 'Setup']
test_list = test_list[test_list['total_call_state'] != 'In-Traffic']
test_list = test_list[test_list['call_type'] != 'U']
test_list.reset_index(drop=True, inplace=True)
test_list['result'] = 0
# ctq_anaysis_merge_list.loc[ctq_anaysis_merge_list['data_ex_rx_thravg'] < 12, 'result'] = 0
test_list.loc[test_list['data_ftp_rx_tp'] < 200, 'result'] = 1

In [23]:
test_drop_list = test_list.drop(['call_id','sampled_time' ,'serving_network','call_type','total_call_state','result','data_ftp_rx_tp','data_ex_rx_thravg'],axis=1)
test_drop_scale = scaler.transform(test_drop_list)
test_y = test_list['result']
test_y[test_y == 1].index

Int64Index([     1,      5,      6,     12,     13,     14,     15,     16,
                17,     20,
            ...
            335748, 335749, 335750, 335756, 335816, 335817, 335818, 335819,
            335820, 335825],
           dtype='int64', length=86524)

In [24]:
test_drop_reshape = test_drop_scale.reshape(-1,TIME_STEPS,INPUT_DIM,1)
test_pred = model1.predict(test_drop_reshape)

In [25]:
from sklearn.metrics import classification_report
# confusion_matrix(train_y, y_pred.argmax(axis=1))
# classfication_report(train_y, y_pred.argmax(axis=1))
print(classification_report(test_y, test_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.96      0.83      0.89    249317
           1       0.65      0.91      0.76     86524

    accuracy                           0.85    335841
   macro avg       0.81      0.87      0.83    335841
weighted avg       0.88      0.85      0.86    335841



In [71]:
conv_name = 'conv2d_5'
grad_cam_total = []
for index, y_data in enumerate(test_y):
    grad_cam_data = test_drop_scale[index]
    grad_cam_data = grad_cam_data.reshape(-1,TIME_STEPS,INPUT_DIM,1)
    grad_cam, grad_val = generate_gradcam(grad_cam_data, model1, y_data, conv_name)
    grad_cam_total.append(grad_cam)
    if (index % 1000) == 0:
        print(index, len(test_y))

    if ((index+1) % 20000) == 0:
        print("종료")
        break;

0 335841
1000 335841
2000 335841
3000 335841
4000 335841
5000 335841
6000 335841
7000 335841
8000 335841
9000 335841
10000 335841
11000 335841
12000 335841
13000 335841
14000 335841
15000 335841
16000 335841
17000 335841
18000 335841
19000 335841
종료


In [81]:
len(grad_cam_total)

20000

In [82]:
np.array(grad_cam_total).argmax(axis=1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [83]:
a= np.array(grad_cam_total)

In [84]:
a=a.reshape(-1,8)
b = a.argmax(axis=1)

In [85]:
pd.Series(b).value_counts()

4    11993
3     6114
5      523
0      443
7      415
1      306
6      135
2       71
dtype: int64

In [86]:
test_sample = test_list[:20000]
sample_pred = test_pred[:20000]

In [87]:
merge_sample = pd.concat([test_sample, pd.DataFrame(sample_pred.argmax(axis=1))],axis=1)

In [92]:
temp = np.array(grad_cam_total)
temp = temp.reshape(-1, 8)

In [94]:
merge_sample = pd.concat([merge_sample, pd.DataFrame(temp)],axis=1)

In [95]:
merge_sample

,call_id,serving_network,sampled_time,call_type,total_call_state,nr5g_Rsrp,nr5g_Sinr,nr5g_TxPwrTotal_Actual,nr5g_Pdsch_Bler,nr5g_pc_pusch_bler,nr5g_DlMcs_Ant0_Avg,nr5g_Rx_NumOfRb_Avg,nr5g_UlMcs_Avg,data_ex_rx_thravg,data_ftp_rx_tp,result,0,0,1,2,3,4,5,6,7
0,20200102082809010348629670036,NRNSA(NR+LTE),2020-01-02 08:28:16,D,T-3ì´ ê³ì°êµ¬ê°,-73.80,2.75,23.0,26.251,3.388,16.077,261.54,25.819,363.128,551.155,0,0,0.000000,0.031127,0.000000,0.168666,0.203635,0.000000,0.000000,0.000000
1,20200102082809010348629670036,NRNSA(NR+LTE),2020-01-02 08:28:17,D,T-3ì´ ê³ì°êµ¬ê°,-70.42,6.66,23.0,13.269,0.433,11.267,270.82,27.219,363.128,187.284,1,0,0.000000,0.000410,0.000000,0.164546,0.177759,0.100673,0.000000,0.000000
2,20200102082809010348629670036,NRNSA(NR+LTE),2020-01-02 08:28:18,D,T-3ì´ ê³ì°êµ¬ê°,-74.51,4.08,23.0,8.833,1.215,16.136,271.85,27.525,363.128,446.872,0,0,0.000000,0.000178,0.000000,0.002072,0.002149,0.000030,0.000000,0.000000
3,20200102082809010348629670036,NRNSA(NR+LTE),2020-01-02 08:28:19,D,T-3ì´ ê³ì°êµ¬ê°,-77.49,5.17,23.0,11.207,0.367,21.885,271.35,27.688,363.128,535.604,0,0,0.000046,0.000045,0.000000,0.000617,0.000655,0.000000,0.000000,0.000000
4,20200102082809010348629670036,NRNSA(NR+LTE),2020-01-02 08:28:20,D,T-3ì´ ê³ì°êµ¬ê°,-78.06,5.29,23.0,9.466,0.012,19.022,271.60,27.382,363.128,493.387,0,0,0.000000,0.000020,0.000000,0.000750,0.000786,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,20200217095636010348626930077,NRNSA(NR+LTE),2020-02-17 09:56:44,D,T-3ì´ ê³ì°êµ¬ê°,-86.64,10.30,8.5,10.996,19.492,14.995,233.00,20.917,457.411,363.387,0,1,0.012902,0.019182,0.014157,0.031817,0.031642,0.010732,0.001094,0.017840
19996,20200217095636010348626930077,NRNSA(NR+LTE),2020-02-17 09:56:45,D,T-3ì´ ê³ì°êµ¬ê°,-89.17,9.70,10.8,10.751,14.025,15.756,246.00,19.653,457.411,485.130,0,0,0.032953,0.027884,0.008406,0.079698,0.078456,0.017333,0.000000,0.027670
19997,20200217095636010348626930077,NRNSA(NR+LTE),2020-02-17 09:56:46,D,T-3ì´ ê³ì°êµ¬ê°,-88.43,9.37,10.8,9.500,11.631,13.853,257.00,18.392,457.411,484.445,0,0,0.011110,0.006218,0.000000,0.038672,0.038100,0.009480,0.000000,0.005406
19998,20200217095636010348626930077,NRNSA(NR+LTE),2020-02-17 09:56:47,D,T-3ì´ ê³ì°êµ¬ê°,-85.99,10.64,8.8,8.954,11.339,14.411,257.00,17.720,457.411,557.662,0,0,0.005898,0.002663,0.000000,0.033499,0.032889,0.005736,0.000000,0.003643


In [98]:
test_file = r'/content/drive/My Drive/WORK/test_result.csv'
merge_sample.to_csv(test_file , encoding='utf8')